In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import csv

import cv2
import numpy as np

from tqdm import tqdm

import os, os.path as osp
from pathlib import Path

from MinkowskiEngine.utils import sparse_quantize

from plyfile import PlyElement, PlyData

from datasets.scannet.common import load_ply

# File names and params

In [ ]:
DATA_DIR = Path('/mnt/data/scannet/scans/')

voxel_size = 0.05

scan_id = 'scene0000_00'
scan_dir = DATA_DIR / scan_id
label_file = DATA_DIR.parent / 'scannetv2-labels.combined.tsv'

input_file = f'{scan_id}_vh_clean_2.ply' 
gt_file = f'{scan_id}_vh_clean_2.labels.ply' 

In [ ]:
# read colors from input file
_, rgb, _ = load_ply(scan_dir / input_file)
# read coords and labels from GT file
coords, _, labels = load_ply(scan_dir / gt_file, read_label=True)
print(coords.dtype, rgb.dtype, labels.dtype)
print(coords.shape, rgb.shape, labels.shape)

print('NYU Labels:', labels.shape, 'Range:', labels.min(), labels.max(), 
      'Unique:', len(np.unique(labels)))
print('coords range', coords.min(axis=0), coords.max(axis=0))

# Voxelize with Minkowski Engine

In [ ]:
coords_vox, rgb_vox, labels_vox, index, inverse = sparse_quantize(coords, rgb, labels, quantization_size=voxel_size, 
                                                                    return_index=True, return_inverse=True, device='cuda')
print('Old->new Index, new->old index', len(index), len(inverse))

unique_coords = coords[index]
print('Unique coords Shape', unique_coords.shape)
print('Unique coords Range', unique_coords.min(axis=0), unique_coords.max(axis=0))
print('Unique coords', unique_coords)

In [ ]:
print('voxelize and ceil/floor unique coords, matches?')
print('Ceil', (np.ceil(unique_coords/voxel_size) == coords_vox).all())
print('Floor', (np.floor(unique_coords/voxel_size) == coords_vox).all())

In [ ]:
print('Shapes', coords_vox.shape, rgb_vox.shape, labels_vox.shape)
print('voxelized coords range', coords_vox.min(axis=0), coords_vox.max(axis=0))
print('Voxelized Dtype', coords_vox.dtype, rgb_vox.dtype, labels_vox.dtype)
print('Label range', labels_vox.min(), labels_vox.max())
print('RGB range', rgb_vox.min(axis=0), rgb_vox.max(axis=0))
print('Voxelized coords', coords_vox)

In [ ]:
# get grid indices from voxelized coords
t = coords_vox.min(axis=0)
coords_new = coords_vox - t
print('Grid indices range', coords_new.min(axis=0), coords_new.max(axis=0))

In [ ]:
(labels_vox == -100).sum()

# Compare coordinates before and after voxelization

In [ ]:
coords.min(axis=0), coords.max(axis=0)

In [ ]:
coords_vox.min(axis=0), coords_vox.max(axis=0)

In [ ]:
coords_vox.min(axis=0) * voxel_size, coords_vox.max(axis=0) * voxel_size

In [ ]:
coords_vox[0]

## save point cloud to file

In [ ]:
# store everything in one array for the PLY file
arr = np.array([tuple(coords_vox[i]) + tuple(rgb_vox[i]) + (labels_vox[i],) for i in range(len(coords_vox))], 
               dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('red', 'f4'), ('green', 'f4'), ('blue', 'f4'), ('label', 'i4')])

elem = PlyElement.describe(arr, 'vertex')

out_file = scan_dir / f'{scan_id}_voxelized.ply'
print(f'Saving to: {out_file}')
PlyData([elem]).write(out_file)

# Read back from file

In [ ]:
c, r, l = load_ply(out_file, read_label=True)
print(c.shape, r.shape, l.shape)
print(c.dtype, r.dtype, l.dtype)
print(c.min(axis=0), c.max(axis=0))
print(r.min(axis=0), r.max(axis=0))
print(l.min(), l.max())